In [1]:
import tensorflow as tf

import INVARIANT
import FLAGS

In [2]:
c = tf.keras.layers.Conv2D(1,3)

In [5]:
c.get_config()

{'name': 'conv2d',
 'trainable': True,
 'dtype': 'float32',
 'filters': 1,
 'kernel_size': (3, 3),
 'strides': (1, 1),
 'padding': 'valid',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'groups': 1,
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'GlorotUniform',
  'config': {'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [ ]:
#vision field table
#<<1024
#3=1+1+1
#5=3+1+1
#7=5+1+1
#9=7+1+1
#10=9+1 << 512
#14=10+2+2
#16=14+2 << 256
#24=16+4+4
#32=24+4+4
#36=32+4 (32=8*4) <<128 -------------OUTPUT  (size: 0-36 pixel)
#52=36+8+8
#68=52+8+8
#76=68+8 (64=16*4) <<64 -------------OUTPUT  (size: 37-76 pixel)
#108=76+16+16
#140=108+16+16
#156=140+16 (128=32*4) <<32 -------------OUTPUT  (size: 77-156 pixel)
#220=156+32+32
#284=220+32+32
#316=284+32 (256=64*4) <<16 -------------OUTPUT  (size: 157-316 pixel)
#444=316+64+64
#572=444+64+64
#572+64=636 (512=128*4) <<8 -------------OUTPUT  (size: 317-636 pixel)
#636+128+128=892 <<8 -------------OUTPUT  (size: 637-892 pixel)
#892+128+128=1020 <<8 -------------OUTPUT  (size: 892-1024 pixel)

In [ ]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))

In [6]:
class ResBlock(tf.keras.layers.Layer):
    def __init__(self,dim,depth):
        super().__init__()
        self.first_layer = tf.keras.layers.Conv2D(dim,3,activation=mish)
        self.rest_layers = [tf.keras.layers.Conv2D(dim,1,activation=mish) for _ in range(depth-1)]
        
    def call(self,inp,training):
        x0 = self.first_layer(inp,training=training)
        x1 = x0
        for l in self.rest_layers:
            x1 = l(x1,training=training)
        output = x0 + x1
        return output

In [8]:
help(tf.keras.layers.ZeroPadding2D)

Help on class ZeroPadding2D in module keras.layers.convolutional:

class ZeroPadding2D(keras.engine.base_layer.Layer)
 |  ZeroPadding2D(*args, **kwargs)
 |  
 |  Zero-padding layer for 2D input (e.g. picture).
 |  
 |  This layer can add rows and columns of zeros
 |  at the top, bottom, left and right side of an image tensor.
 |  
 |  Examples:
 |  
 |  >>> input_shape = (1, 1, 2, 2)
 |  >>> x = np.arange(np.prod(input_shape)).reshape(input_shape)
 |  >>> print(x)
 |  [[[[0 1]
 |     [2 3]]]]
 |  >>> y = tf.keras.layers.ZeroPadding2D(padding=1)(x)
 |  >>> print(y)
 |  tf.Tensor(
 |    [[[[0 0]
 |       [0 0]
 |       [0 0]
 |       [0 0]]
 |      [[0 0]
 |       [0 1]
 |       [2 3]
 |       [0 0]]
 |      [[0 0]
 |       [0 0]
 |       [0 0]
 |       [0 0]]]], shape=(1, 3, 4, 2), dtype=int64)
 |  
 |  Args:
 |    padding: Int, or tuple of 2 ints, or tuple of 2 tuples of 2 ints.
 |      - If int: the same symmetric padding
 |        is applied to height and width.
 |      - If tuple of

In [ ]:
class Block(tf.keras.layers.Layer):
    def __init__(self,dim,subblocks,depth):
        self.padding = tf.keras.layers.ZeroPadding2D(subblocks)
        self.subblocks = [ResBlock(dim,depth) for _ in range(subblocks)]
    def call(self,inp,training):
        

In [ ]:
FLAGS.DATA.TRAIN.LABEL_FOLDER = r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\label\custom'
FLAGS.DATA.TRAIN.IMAGE_FOLDER = r'C:\Users\Atlas\Desktop\Sinalboard_OCR\dataset\train\image\official'
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RATIO = 0.05
FLAGS.DATA.TRAIN.VALIDATION_SPLIT_RANDOM_STATE = 100
FLAGS.DATA.TRAIN.BATCH_SIZE = 8
FLAGS.DATA.TEST = edict()
FLAGS.DATA.TEST.IMAGE_PATH = None
FLAGS.DATA.TEST.BATCH_SIZE = None

MODEL = None

OPTIMIZER = edict()
OPTIMIZER.TYPE = None
OPTIMIZER.MAX_LEARNING_RATE = None
OPTIMIZER.SCHEDULE_GAMMA = None

EPOCHS = edict()
EPOCHS.TOTAL = None
EPOCHS.WARMUP = None

LOGGING = edict()
LOGGING.PATH = None
LOGGING.MODEL_NAME = None
LOGGING.TRIAL_NUMBER = None
LOGGING.NOTE = None
LOGGING.SAMPLES_PER_LOG = None
LOGGING.TEST_IMAGE_COLUMNS = None